In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define dataset directories
DATASET_DIR = "Planet/planet"
TRAIN_DIR = os.path.join(DATASET_DIR, "train-jpg")
TEST_DIR = os.path.join(DATASET_DIR, "test-jpg")

# Optimized parameters for weak CPU
IMG_SIZE = (128, 128)  # Adjusted for efficiency
BATCH_SIZE = 8         # Reduced batch size to lower memory usage
EPOCHS = 10            # Adjusted epochs for balance between accuracy and speed

# Define a CNN model
def build_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Preprocess dataset with memory optimization
def preprocess_dataset(input_dir, limit=None):
    images = []
    labels = []
    image_files = [f for f in os.listdir(input_dir) if f.endswith(".jpg")]
    if limit:
        image_files = image_files[:limit]  # Limit dataset to reduce memory usage
    for img_file in tqdm(image_files, desc=f"Processing {input_dir}"):
        img_path = os.path.join(input_dir, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, IMG_SIZE)
        img = img / 255.0
        images.append(img)
        labels.append(0)  # Placeholder label
    return np.array(images), np.array(labels)

# Train the model with memory efficiency
def train_model():
    train_images, train_labels = preprocess_dataset(TRAIN_DIR, limit=1000)  # Limit training data
    model = build_model()
    model.fit(train_images, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)
    model.save(os.path.join(DATASET_DIR, "deforestation_model.h5"))
    print("Model training complete!")

# Test model on a randomly selected test image
def test_model():
    model_path = os.path.join(DATASET_DIR, "deforestation_model.h5")
    if not os.path.exists(model_path):
        print("Error: Model file not found! Training the model first...")
        train_model()
    model = tf.keras.models.load_model(model_path)
    
    image_files = [f for f in os.listdir(TEST_DIR) if f.endswith(".jpg")]
    if not image_files:
        print("No test images found!")
        return
    
    img_file = random.choice(image_files)  # Select a random test image
    img_path = os.path.join(TEST_DIR, img_file)
    img = cv2.imread(img_path)
    if img is None:
        print("Error: Could not read image!")
        return
    
    img_resized = cv2.resize(img, IMG_SIZE) / 255.0
    img_resized = np.expand_dims(img_resized, axis=0)
    
    prediction = model.predict(img_resized)[0][0]
    
    plt.figure(figsize=(5, 5))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Deforestation Probability: {prediction:.2f}\n({img_file})")
    plt.show()

def main():
    print("Deforestation detection system ready!")
    train_model()
    test_model()

if __name__ == "__main__":
    main()


Deforestation detection system ready!


Processing Planet/planet\train-jpg: 100%|██████████| 1000/1000 [00:01<00:00, 594.06it/s]
C:\Users\Moon-Study\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 115ms/step - accuracy: 0.9807 - loss: 0.0356 - val_accuracy: 1.0000 - val_loss: 4.9992e-22
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 116ms/step - accuracy: 1.0000 - loss: 2.5745e-17 - val_accuracy: 1.0000 - val_loss: 4.9799e-22
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 116ms/step - accuracy: 1.0000 - loss: 8.9125e-20 - val_accuracy: 1.0000 - val_loss: 4.9799e-22
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 1.0000 - loss: 2.9388e-18 - val_accuracy: 1.0000 - val_loss: 4.9799e-22
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - accuracy: 1.0000 - loss: 1.1312e-18 - val_accuracy: 1.0000 - val_loss: 4.9799e-22
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - accuracy: 1.0000 - loss: 3.7692e-20 - val_accuracy: 1.0000 - val_loss: 4.9799e-22
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - accuracy: 1.0000 - loss: 5.2793e-18 - val_accuracy: 1.0000 - val_loss: 4.9799e-22
Epoch 8/10
100/100 ━━━━━━━━━━━━